In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
from PIL import Image
from utils import read_mat_file, visualize_sample_dataset, vae_loss
from dataloader import SVHDDataset, SVHDDigitOnlyDigitDataset
import scipy.io

from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from model import VGG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
import os
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
# Train the model
from model import VGG16, VAE
import cv2 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image

os.environ['TORCH_NNPACK'] = '0'

In [2]:
train_path = '/data/omscs_datasets/train/'
train_mat_file_path = train_path + 'digitStruct.mat'
test_path = '/data/omscs_datasets/train/'
test_mat_file_path = test_path + 'digitStruct.mat'
device = torch.device("cuda")

In [3]:
train_dataset = SVHDDigitOnlyDigitDataset(train_mat_file_path, train_path,mode='train')

## train ##


In [4]:
train_images = []
train_labels = []
for t in tqdm(train_dataset):
    for tt in t:
        train_images.append(tt['image'])
        train_labels.append(tt['label'])

33402it [02:57, 188.13it/s]          


In [5]:
train_images[0]

tensor([[[0.3412, 0.3216, 0.3098,  ..., 0.1882, 0.2745, 0.2902],
         [0.3176, 0.3216, 0.3255,  ..., 0.1961, 0.2745, 0.2902],
         [0.3294, 0.3255, 0.3176,  ..., 0.2314, 0.2863, 0.3020],
         ...,
         [0.4000, 0.4157, 0.4314,  ..., 0.3529, 0.3294, 0.3216],
         [0.3882, 0.4078, 0.4235,  ..., 0.3922, 0.3765, 0.3647],
         [0.3804, 0.3961, 0.4196,  ..., 0.3961, 0.3843, 0.3882]],

        [[0.2902, 0.2902, 0.2980,  ..., 0.1725, 0.2392, 0.2824],
         [0.3137, 0.3098, 0.3059,  ..., 0.1922, 0.2471, 0.2745],
         [0.3176, 0.3216, 0.3137,  ..., 0.2157, 0.2627, 0.2902],
         ...,
         [0.4039, 0.3961, 0.3882,  ..., 0.3098, 0.3137, 0.3255],
         [0.4196, 0.4118, 0.4078,  ..., 0.3765, 0.3804, 0.3765],
         [0.4196, 0.4157, 0.4196,  ..., 0.4118, 0.4196, 0.4196]],

        [[0.2392, 0.2549, 0.2471,  ..., 0.1569, 0.1765, 0.1804],
         [0.2314, 0.2431, 0.2314,  ..., 0.1608, 0.2000, 0.1961],
         [0.2196, 0.2157, 0.2275,  ..., 0.1686, 0.2039, 0.

In [6]:
# 이미지 텐서를 쌓습니다
stacked_images = torch.stack(train_images, dim=0)

# 채널별 평균과 표준편차를 계산합니다
mean = torch.mean(stacked_images, dim=[0, 2, 3])
std = torch.std(stacked_images, dim=[0, 2, 3])

print(f"Calculated mean: {mean}")
print(f"Calculated std: {std}")

Calculated mean: tensor([0.4342, 0.4431, 0.4768])
Calculated std: tensor([0.1927, 0.1956, 0.1931])


In [7]:
train_transform = transforms.Compose([
    #transforms.Resize((32, 32)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.4342, 0.4431, 0.4768], std=[0.1927, 0.1956, 0.1931])  # 일반적인 ImageNet 평균 및 표준편차

])

from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        sample = {'image': image, 'label': label}
        return sample

In [8]:
print(len(train_images), len(train_labels))
train_dataset = CustomDataset(train_images, train_labels, transform=train_transform)

73257 73257


In [9]:
print(len(train_images), len(train_labels))
train_dataset = CustomDataset(train_images, train_labels, transform=train_transform)

73257 73257


In [10]:
# Specify the lengths of the training and testing sets
train_size = int(1 * len(train_dataset))
test_size = len(train_dataset) - train_size

# Use random_split to split the dataset
train_data, test_data = random_split(train_dataset, [train_size, test_size])
print(f'train_data:{len(train_data)}, test_data:{len(test_data)}')

train_data:73257, test_data:0


In [11]:
# Step 4: Create a DataLoader for your dataset
train_loader = DataLoader(train_data, batch_size=216, shuffle=True)
test_loader = DataLoader(test_data, batch_size=216, shuffle=False)
print(len(train_loader), len(test_loader))

340 0


In [12]:
def vae_loss(reconstructed_x, x, mu, log_var):
    reconstruction_loss = F.binary_cross_entropy(reconstructed_x, x, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return reconstruction_loss + kl_divergence

In [15]:
def save_reconstructed_images(reconstructed, original, epoch, folder="reconstructed_images"):
    os.makedirs(folder, exist_ok=True)
    # 이미지 정규화 해제
    reconstructed = reconstructed #* 255.
    original = original #* 255.

    # 이미지 저장
    save_image(reconstructed, f'{folder}/reconstructed_epoch_{epoch}.png')
    save_image(original, f'{folder}/original_epoch_{epoch}.png')

In [18]:
# Hyperparameters
image_channels = 3
h_dim = 1024
z_dim = 32
learning_rate = 1e-3
os.system('rm -rf ./reconstructed_images/*')
# 모델 및 옵티마이저 초기화
vae = VAE(image_channels=image_channels, h_dim=h_dim, z_dim=z_dim)
vae = vae.to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)
rec_loss = np.inf

# 학습 루프
for epoch in range(300):
    for batch in tqdm(train_loader):
        x, labels = batch['image'], batch['label']
        x = x.to(torch.float32).to(device)
        x = x.to(device)
        x_reconstructed, mu, log_var = vae(x)
        #print('x:', x)
        #print('x_reconstructed:', x_reconstructed)
        loss = vae_loss(x_reconstructed, x, mu, log_var)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if loss < rec_loss:
        torch.save(vae.state_dict(),'weights/vae_best_model.pth')
    print(f'Epoch:{epoch} Reconstruction Loss:{loss}')
    if epoch % 10 == 0:  # 예를 들어, 매 50 에포크마다 이미지 저장
            save_reconstructed_images(x_reconstructed, x, epoch)

100%|██████████| 340/340 [00:02<00:00, 122.01it/s]


Epoch:0 Reconstruction Loss:63913.69921875


100%|██████████| 340/340 [00:02<00:00, 137.63it/s]


Epoch:1 Reconstruction Loss:63525.4453125


100%|██████████| 340/340 [00:02<00:00, 133.79it/s]


...

Epoch:297 Reconstruction Loss:63209.0390625


100%|██████████| 340/340 [00:02<00:00, 131.96it/s]


Epoch:298 Reconstruction Loss:62697.78125


100%|██████████| 340/340 [00:02<00:00, 130.64it/s]

Epoch:299 Reconstruction Loss:60517.88671875


In [20]:
vae.load_state_dict(torch.load('weights/vae_best_model.pth'))

<All keys matched successfully>